# YouTube Tag Effectiveness Analysis

This notebook analyzes the effectiveness of YouTube video tags based on various performance metrics like views, likes, comments, and subscriber acquisition.

## Features:
- Tag performance visualization
- Interactive filtering by show
- Tag exclusion capabilities
- Multiple performance metrics analysis
- Statistical insights

In [32]:
# Import required libraries
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, clear_output
import warnings
from collections import Counter
import re

warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")

print("Libraries imported successfully!")

Libraries imported successfully!


In [33]:
# Connect to database
db_path = '../data/youtube_analytics.sqlite'
conn = sqlite3.connect(db_path)

# Load video stats
query = """
SELECT
    video_id,
    title,
    description,
    upload_time,
    duration,
    view_count,
    like_count,
    comment_count,
    visibility,
    is_short,
    show,
    episode_num,
    tags,
    subscriber_count,
    collected_at,
    last_updated
FROM video_stats
WHERE exclude_from_stats = 0
  AND visibility = 'public'
  AND tags IS NOT NULL
  AND tags != ''
  AND episode_num >= 127
ORDER BY upload_time DESC
"""

df = pd.read_sql_query(query, conn)
conn.close()

# Convert dates and handle timezone issues
df['upload_time'] = pd.to_datetime(df['upload_time'], utc=True)
df['collected_at'] = pd.to_datetime(df['collected_at'], utc=True)

# Calculate performance metrics
df['like_rate'] = df['like_count'] / df['view_count'].replace(0, np.nan)
df['comment_rate'] = df['comment_count'] / df['view_count'].replace(0, np.nan)
df['engagement_rate'] = (df['like_count'] + df['comment_count']) / df['view_count'].replace(0, np.nan)

# Handle subscriber count (fill NaN with 0 for analysis)
df['subscriber_count'] = df['subscriber_count'].fillna(0)
df['subscriber_rate'] = df['subscriber_count'] / df['view_count'].replace(0, np.nan)

# Duration in minutes for easier understanding
df['duration_minutes'] = df['duration'] / 60

# Create age in days (handle timezone-aware comparison)
current_time = pd.Timestamp.now(tz='UTC')
df['age_days'] = (current_time - df['upload_time']).dt.days

print(f"Loaded {len(df)} videos with tags")
print(f"Date range: {df['upload_time'].min().date()} to {df['upload_time'].max().date()}")
print(f"Shows available: {sorted(df['show'].dropna().unique())}")


df.head()

Loaded 91 videos with tags
Date range: 2024-01-01 to 2025-09-29
Shows available: ['DOU News']


,video_id,title,description,upload_time,duration,view_count,like_count,comment_count,visibility,is_short,...,tags,subscriber_count,collected_at,last_updated,like_rate,comment_rate,engagement_rate,subscriber_rate,duration_minutes,age_days
0,pntx8StorzM,Чи був «злив» даних з «Дії» | Нові блокування ...,📰 У свіжому дайджесті новин поговоримо про зав...,2025-09-29 10:01:26+00:00,1887,9603,1035,63,public,0,...,"dou, доу, DOU News, новини IT, українське IT, ...",0,2025-09-29 21:02:54+00:00,2025-09-29 21:17:37,0.107779,0.006560,0.114339,0.000000,31.450000,0
1,rHb4zI-IWfE,Портрет айтівця 2025 | Реклама вже на холодиль...,📰 У свіжому дайджесті новин поговоримо про рек...,2025-09-22 10:01:49+00:00,1774,26093,1087,57,public,0,...,"dou, доу, Samsung реклама холодильники, Meta с...",8,2025-09-29 21:03:08+00:00,2025-09-29 21:17:47,0.041659,0.002184,0.043843,0.000307,29.566667,7
2,lWUeEYUbots,Виробників дронів бронюють | ШІ вже член уряду...,"👉Столи з електрорегулюванням висоти, крісла та...",2025-09-15 10:01:19+00:00,1790,13035,1145,47,public,0,...,"dou, доу, news, google, it, yjdbyb, lje, ueuk,...",14,2025-09-29 21:04:48+00:00,2025-09-29 21:22:47,0.087840,0.003606,0.091446,0.001074,29.833333,14
3,hgG8mY8LS84,Цифровий офіцер | Факультет дронів у КАІ | Atl...,📰 У свіжому дайджесті DOU News поговоримо про ...,2025-09-08 10:01:15+00:00,1518,27298,1129,51,public,0,...,"dou, доу, news, google, it, Anthropic, eu, Atl...",32,2025-09-09 15:34:20+00:00,2025-09-29 21:17:57,0.041358,0.001868,0.043227,0.001172,25.300000,21
4,XG0v_MAV0ys,Команда 🇺🇦 LLM | DOU Day Picnic 2025 | Возняк ...,📰 У свіжому дайджесті новин обговорюємо команд...,2025-09-01 10:00:56+00:00,1441,29881,1302,58,public,0,...,"dou, доу, ai, llm, dou day, dou news",20,2025-09-09 15:34:20+00:00,2025-09-29 21:18:03,0.043573,0.001941,0.045514,0.000669,24.016667,28


In [34]:
def process_tags(df, exclude_tags=None):
    """Process tags and create tag analysis dataset"""

    if exclude_tags is None:
        exclude_tags = []

    # Convert to lowercase for consistent comparison
    exclude_tags_lower = [tag.lower().strip() for tag in exclude_tags]

    tag_data = []

    for idx, row in df.iterrows():
        if pd.isna(row['tags']) or row['tags'] == '':
            continue

        # Split tags and clean them
        tags = [tag.strip() for tag in row['tags'].split(',') if tag.strip()]

        for tag in tags:
            # Skip excluded tags (case-insensitive)
            if tag.lower() in exclude_tags_lower:
                continue

            tag_data.append({
                'video_id': row['video_id'],
                'title': row['title'],
                'tag': tag,
                'tag_lower': tag.lower(),
                'show': row['show'],
                'episode_num': row['episode_num'],
                'is_short': row['is_short'],
                'upload_time': row['upload_time'],
                'age_days': row['age_days'],
                'duration_minutes': row['duration_minutes'],
                'view_count': row['view_count'],
                'like_count': row['like_count'],
                'comment_count': row['comment_count'],
                'subscriber_count': row['subscriber_count'],
                'like_rate': row['like_rate'],
                'comment_rate': row['comment_rate'],
                'engagement_rate': row['engagement_rate'],
                'subscriber_rate': row['subscriber_rate']
            })

    tag_df = pd.DataFrame(tag_data)

    print(f"Processed {len(tag_df)} tag instances from {len(df)} videos")
    if exclude_tags:
        print(f"Excluded tags: {exclude_tags}")

    return tag_df

def get_tag_performance(tag_df, min_usage=3):
    """Calculate tag performance metrics"""

    # Group by tag and calculate metrics
    tag_stats = tag_df.groupby('tag').agg({
        'video_id': 'count',  # usage count
        'view_count': ['mean', 'median', 'sum'],
        'like_count': ['mean', 'median', 'sum'],
        'comment_count': ['mean', 'median', 'sum'],
        'subscriber_count': ['mean', 'median', 'sum'],
        'like_rate': ['mean', 'median'],
        'comment_rate': ['mean', 'median'],
        'engagement_rate': ['mean', 'median'],
        'subscriber_rate': ['mean', 'median'],
        'duration_minutes': 'mean',
        'age_days': 'mean'
    }).round(4)

    # Flatten column names
    tag_stats.columns = ['_'.join(col).strip() for col in tag_stats.columns]
    tag_stats = tag_stats.rename(columns={'video_id_count': 'usage_count'})

    # Filter by minimum usage
    tag_stats = tag_stats[tag_stats['usage_count'] >= min_usage]

    # Calculate effectiveness scores (normalized)
    metrics = ['view_count_mean', 'like_rate_mean', 'engagement_rate_mean', 'subscriber_rate_mean']

    for metric in metrics:
        if metric in tag_stats.columns:
            # Normalize to 0-100 scale
            max_val = tag_stats[metric].max()
            min_val = tag_stats[metric].min()
            if max_val > min_val:
                tag_stats[f'{metric}_normalized'] = 100 * (tag_stats[metric] - min_val) / (max_val - min_val)
            else:
                tag_stats[f'{metric}_normalized'] = 50

    # Overall effectiveness score (weighted average)
    weights = {'view_count_mean_normalized': 0.4, 'engagement_rate_mean_normalized': 0.3,
              'subscriber_rate_mean_normalized': 0.2, 'like_rate_mean_normalized': 0.1}

    tag_stats['effectiveness_score'] = 0
    for metric, weight in weights.items():
        if metric in tag_stats.columns:
            tag_stats['effectiveness_score'] += tag_stats[metric] * weight

    # Sort by effectiveness
    tag_stats = tag_stats.sort_values('effectiveness_score', ascending=False)

    print(f"Analysis complete for {len(tag_stats)} tags (min usage: {min_usage})")

    return tag_stats

# Initial tag processing
tag_df = process_tags(df)
tag_stats = get_tag_performance(tag_df)

print("\\nTop 10 most used tags:")
print(tag_stats['usage_count'].head(10))

Processed 2898 tag instances from 91 videos
Analysis complete for 122 tags (min usage: 3)
\nTop 10 most used tags:
tag
Revolut            3
фоп 3 група        6
фоп                7
дія бронювання     3
Київстар           6
бронювання it      3
приват24           3
приватбанк         3
GPT-4             14
влк               13
Name: usage_count, dtype: int64


In [35]:
# Analysis type
analysis_type = widgets.Dropdown(
    options=[
        ('Tag Effectiveness Overview', 'overview'),
        ('Views Performance', 'views'),
        ('Engagement Analysis', 'engagement'),
        ('Subscriber Acquisition', 'subscribers'),
        ('Usage vs Total Viewership', 'usage_vs_views'),
        ('Tag Usage Patterns', 'usage')
    ],
    value='overview',
    description='Analysis Type:',
    style={'description_width': 'initial'}
)

In [36]:
def create_tag_usage_vs_viewership(tag_df, tag_stats, top_n=50):
    """Create scatter plot showing tag usage count vs total viewership"""

    # Calculate total viewership for each tag
    tag_viewership = tag_df.groupby('tag')['view_count'].sum().reset_index()
    tag_viewership.columns = ['tag', 'total_views']

    # Merge with usage count from tag_stats
    plot_data = tag_viewership.merge(
        tag_stats['usage_count'].reset_index(),
        on='tag',
        how='inner'
    )

    # Get top N tags by total viewership for cleaner visualization
    plot_data = plot_data.nlargest(top_n, 'total_views')

    # Create the scatter plot
    fig = go.Figure()

    fig.add_trace(
        go.Scatter(
            x=plot_data['total_views'],
            y=plot_data['usage_count'],
            mode='markers+text',
            marker=dict(
                size=12,
                color=plot_data['usage_count'],
                colorscale='Viridis',
                showscale=True,
                colorbar=dict(title="Usage Count"),
                opacity=0.7
            ),
            text=plot_data['tag'],
            textposition='top center',
            textfont=dict(size=10),
            hovertemplate=(
                '<b>%{text}</b><br>' +
                'Total Views: %{x:,.0f}<br>' +
                'Usage Count: %{y}<br>' +
                'Avg Views per Video: %{customdata:,.0f}<extra></extra>'
            ),
            customdata=plot_data['total_views'] / plot_data['usage_count']
        )
    )

    # Add trend line
    from scipy import stats
    if len(plot_data) > 2:
        slope, intercept, r_value, p_value, std_err = stats.linregress(
            plot_data['total_views'],
            plot_data['usage_count']
        )

        x_trend = np.linspace(plot_data['total_views'].min(), plot_data['total_views'].max(), 100)
        y_trend = slope * x_trend + intercept

        fig.add_trace(
            go.Scatter(
                x=x_trend,
                y=y_trend,
                mode='lines',
                name=f'Trend Line (R² = {r_value**2:.3f})',
                line=dict(color='red', dash='dash', width=2),
                hovertemplate=f'Trend Line<br>R² = {r_value**2:.3f}<extra></extra>'
            )
        )

    fig.update_layout(
        title=f'Tag Usage Count vs Total Viewership (Top {top_n} Tags)',
        xaxis_title='Total Views Across All Videos',
        yaxis_title='Number of Times Tag Was Used',
        height=600,
        showlegend=True
    )

    # Format x-axis to show large numbers nicely
    fig.update_xaxes(tickformat=',.0f')

    return fig

def create_tag_efficiency_analysis(tag_df, tag_stats, top_n=30):
    """Create analysis showing tag efficiency (views per usage)"""

    # Calculate total viewership and efficiency for each tag
    tag_analysis = tag_df.groupby('tag').agg({
        'view_count': ['sum', 'mean', 'count']
    }).round(0)

    # Flatten column names
    tag_analysis.columns = ['total_views', 'avg_views_per_video', 'usage_count']
    tag_analysis['views_per_usage'] = tag_analysis['total_views'] / tag_analysis['usage_count']

    # Filter by minimum usage and get top performers
    tag_analysis = tag_analysis[tag_analysis['usage_count'] >= 2]
    top_tags = tag_analysis.nlargest(top_n, 'total_views')

    # Create subplot with dual analysis
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('Usage vs Total Views', 'Tag Efficiency (Views per Usage)'),
        specs=[[{"secondary_y": False}, {"secondary_y": False}]]
    )

    # Left plot: Usage vs Total Views
    fig.add_trace(
        go.Scatter(
            x=top_tags['total_views'],
            y=top_tags['usage_count'],
            mode='markers+text',
            text=top_tags.index,
            textposition='top center',
            marker=dict(
                size=10,
                color='lightblue',
                line=dict(color='darkblue', width=1)
            ),
            name='Tags',
            hovertemplate=(
                '<b>%{text}</b><br>' +
                'Total Views: %{x:,.0f}<br>' +
                'Usage Count: %{y}<extra></extra>'
            )
        ),
        row=1, col=1
    )

    # Right plot: Efficiency analysis
    efficiency_top = top_tags.nlargest(20, 'views_per_usage')
    fig.add_trace(
        go.Bar(
            x=efficiency_top.index,
            y=efficiency_top['views_per_usage'],
            marker_color='lightgreen',
            name='Efficiency',
            hovertemplate=(
                '<b>%{x}</b><br>' +
                'Views per Usage: %{y:,.0f}<br>' +
                'Total Uses: %{customdata}<extra></extra>'
            ),
            customdata=efficiency_top['usage_count']
        ),
        row=1, col=2
    )

    fig.update_layout(
        title='Tag Usage Analysis: Volume vs Efficiency',
        height=600,
        showlegend=False
    )

    # Format axes
    fig.update_xaxes(tickformat=',.0f', title_text="Total Views", row=1, col=1)
    fig.update_yaxes(title_text="Usage Count", row=1, col=2)
    fig.update_xaxes(tickangle=45, title_text="Tag", row=1, col=2)
    fig.update_yaxes(title_text="Views per Usage", row=1, col=2)

    return fig
global filtered_tag_df
global filtered_tag_stats
print("Tag usage vs viewership visualization functions added!")

Tag usage vs viewership visualization functions added!


In [37]:
# Create interactive widgets
def create_analysis_controls():
    """Create interactive controls for tag analysis"""
    
    # Show filter
    available_shows = ['All Shows'] + sorted([show for show in df['show'].dropna().unique() if show])
    show_filter = widgets.Dropdown(
        options=available_shows,
        value='All Shows',
        description='Show Filter:',
        style={'description_width': 'initial'}
    )
    
    # Tag exclusion
    exclude_tags_input = widgets.Textarea(
        value='',
        placeholder='Enter tags to exclude (comma-separated)\nExample: music, background music, intro',
        description='Exclude Tags:',
        rows=3,
        style={'description_width': 'initial'}
    )
    
    # Minimum usage threshold
    min_usage_slider = widgets.IntSlider(
        value=3,
        min=1,
        max=20,
        step=1,
        description='Min Tag Usage:',
        style={'description_width': 'initial'}
    )
    
    # Analysis type
    analysis_type = widgets.Dropdown(
        options=[
            ('Tag Effectiveness Overview', 'overview'),
            ('Views Performance', 'views'),
            ('Engagement Analysis', 'engagement'),
            ('Subscriber Acquisition', 'subscribers'),
            ('Tag Usage Patterns', 'usage')
        ],
        value='overview',
        description='Analysis Type:',
        style={'description_width': 'initial'}
    )
    
    # Update button
    update_button = widgets.Button(
        description='Update Analysis',
        button_style='primary',
        tooltip='Click to refresh analysis with current settings'
    )
    
    return {
        'show_filter': show_filter,
        'exclude_tags_input': exclude_tags_input,
        'min_usage_slider': min_usage_slider,
        'analysis_type': analysis_type,
        'update_button': update_button
    }

controls = create_analysis_controls()

# Display controls
print("Analysis Controls:")
display(widgets.VBox([
    widgets.HBox([controls['show_filter'], controls['analysis_type']]),
    controls['exclude_tags_input'],
    controls['min_usage_slider'],
    controls['update_button']
]))

Analysis Controls:


In [38]:
def create_tag_effectiveness_overview(tag_stats, top_n=20):
    """Create comprehensive tag effectiveness visualization"""
    
    top_tags = tag_stats.head(top_n)
    
    # Create subplots
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Tag Effectiveness Score',
            'Average Views per Video',
            'Engagement Rate',
            'Subscriber Acquisition Rate'
        ),
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}]]
    )
    
    # Effectiveness Score
    fig.add_trace(
        go.Bar(
            x=top_tags.index,
            y=top_tags['effectiveness_score'],
            name='Effectiveness Score',
            marker_color='lightblue',
            text=top_tags['usage_count'].astype(str),
            textposition='outside',
            hovertemplate='<b>%{x}</b><br>Score: %{y:.1f}<br>Usage: %{text} videos<extra></extra>'
        ),
        row=1, col=1
    )
    
    # Average Views
    fig.add_trace(
        go.Bar(
            x=top_tags.index,
            y=top_tags['view_count_mean'],
            name='Avg Views',
            marker_color='lightgreen',
            hovertemplate='<b>%{x}</b><br>Avg Views: %{y:,.0f}<extra></extra>'
        ),
        row=1, col=2
    )
    
    # Engagement Rate
    fig.add_trace(
        go.Bar(
            x=top_tags.index,
            y=top_tags['engagement_rate_mean'] * 100,  # Convert to percentage
            name='Engagement Rate (%)',
            marker_color='lightcoral',
            hovertemplate='<b>%{x}</b><br>Engagement: %{y:.2f}%<extra></extra>'
        ),
        row=2, col=1
    )
    
    # Subscriber Rate
    fig.add_trace(
        go.Bar(
            x=top_tags.index,
            y=top_tags['subscriber_rate_mean'] * 100,  # Convert to percentage
            name='Subscriber Rate (%)',
            marker_color='gold',
            hovertemplate='<b>%{x}</b><br>Subscriber Rate: %{y:.3f}%<extra></extra>'
        ),
        row=2, col=2
    )
    
    # Update layout
    fig.update_layout(
        title=f'Top {top_n} Most Effective YouTube Tags',
        height=800,
        showlegend=False
    )
    
    # Rotate x-axis labels
    fig.update_xaxes(tickangle=45)
    
    return fig

def create_tag_scatter_analysis(tag_stats, x_metric, y_metric, size_metric='usage_count', top_n=30):
    """Create scatter plot analysis for tag performance"""
    
    top_tags = tag_stats.head(top_n)
    
    # Metric display names
    metric_names = {
        'view_count_mean': 'Average Views',
        'engagement_rate_mean': 'Engagement Rate',
        'subscriber_rate_mean': 'Subscriber Rate',
        'usage_count': 'Usage Count',
        'effectiveness_score': 'Effectiveness Score'
    }
    
    x_values = top_tags[x_metric]
    y_values = top_tags[y_metric]
    
    # Convert rates to percentages for display
    if 'rate' in x_metric:
        x_values = x_values * 100
    if 'rate' in y_metric:
        y_values = y_values * 100
    
    fig = go.Figure()
    
    fig.add_trace(
        go.Scatter(
            x=x_values,
            y=y_values,
            mode='markers+text',
            marker=dict(
                size=top_tags[size_metric],
                sizemode='diameter',
                sizeref=2.*max(top_tags[size_metric])/(40.**2),
                sizemin=4,
                color=top_tags['effectiveness_score'],
                colorscale='Viridis',
                showscale=True,
                colorbar=dict(title="Effectiveness Score")
            ),
            text=top_tags.index,
            textposition='top center',
            hovertemplate=(
                '<b>%{text}</b><br>' +
                f'{metric_names.get(x_metric, x_metric)}: %{{x:.2f}}<br>' +
                f'{metric_names.get(y_metric, y_metric)}: %{{y:.2f}}<br>' +
                f'{metric_names.get(size_metric, size_metric)}: %{{marker.size}}<br>' +
                'Effectiveness: %{marker.color:.1f}<extra></extra>'
            )
        )
    )
    
    fig.update_layout(
        title=f'{metric_names.get(y_metric, y_metric)} vs {metric_names.get(x_metric, x_metric)}',
        xaxis_title=metric_names.get(x_metric, x_metric),
        yaxis_title=metric_names.get(y_metric, y_metric),
        height=600
    )
    
    return fig

def create_tag_usage_analysis(tag_df, top_n=20):
    """Analyze tag usage patterns over time and by show"""
    
    # Most used tags
    tag_usage = tag_df['tag'].value_counts().head(top_n)
    
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=('Most Used Tags', 'Tag Usage by Show'),
        row_heights=[0.6, 0.4]
    )
    
    # Top tags bar chart
    fig.add_trace(
        go.Bar(
            x=tag_usage.index,
            y=tag_usage.values,
            name='Usage Count',
            marker_color='skyblue',
            hovertemplate='<b>%{x}</b><br>Used in %{y} videos<extra></extra>'
        ),
        row=1, col=1
    )
    
    # Tag usage by show (if show data available)
    if 'show' in tag_df.columns and tag_df['show'].notna().any():
        show_tag_usage = tag_df[tag_df['show'].notna()].groupby('show')['tag'].count().sort_values(ascending=True)
        
        fig.add_trace(
            go.Bar(
                x=show_tag_usage.values,
                y=show_tag_usage.index,
                orientation='h',
                name='Tags per Show',
                marker_color='lightgreen',
                hovertemplate='<b>%{y}</b><br>Total tags: %{x}<extra></extra>'
            ),
            row=2, col=1
        )
    
    fig.update_layout(
        title='Tag Usage Analysis',
        height=800,
        showlegend=False
    )
    
    fig.update_xaxes(tickangle=45, row=1, col=1)
    
    return fig

print("Visualization functions loaded successfully!")

Visualization functions loaded successfully!


## Interactive Analysis

In [39]:
# Output area for analysis results
output_area = widgets.Output()

def run_analysis():
    """Run the analysis based on current control settings"""

    with output_area:
        clear_output(wait=True)
        
        # Get current settings
        show_filter = controls['show_filter'].value
        exclude_tags_text = controls['exclude_tags_input'].value
        min_usage = controls['min_usage_slider'].value
        analysis_type = controls['analysis_type'].value
        
        # Parse excluded tags
        exclude_tags = [tag.strip() for tag in exclude_tags_text.split(',') if tag.strip()]
        
        # Filter data by show
        filtered_df = df.copy()
        if show_filter != 'All Shows':
            filtered_df = filtered_df[filtered_df['show'] == show_filter]
        
        print(f"Analysis Settings:")
        print(f"- Show Filter: {show_filter}")
        print(f"- Excluded Tags: {exclude_tags if exclude_tags else 'None'}")
        print(f"- Minimum Usage: {min_usage}")
        print(f"- Videos in Analysis: {len(filtered_df)}")
        print("\n" + "="*50 + "\n")
        
        if len(filtered_df) == 0:
            print("No data available with current filters!")
            return
        
        # Process tags with current settings
        filtered_tag_df = process_tags(filtered_df, exclude_tags)
        filtered_tag_stats = get_tag_performance(filtered_tag_df, min_usage)
        
        if len(filtered_tag_stats) == 0:
            print(f"No tags meet the minimum usage threshold of {min_usage}!")
            return
        
        # Generate appropriate visualization
        if analysis_type == 'overview':
            fig = create_tag_effectiveness_overview(filtered_tag_stats)
            fig.show()
            
            # Show top performing tags summary
            print("\nTop 10 Most Effective Tags:")
            summary_cols = ['usage_count', 'view_count_mean', 'engagement_rate_mean', 'subscriber_rate_mean', 'effectiveness_score']
            summary_df = filtered_tag_stats[summary_cols].head(10)
            summary_df['engagement_rate_mean'] *= 100  # Convert to percentage
            summary_df['subscriber_rate_mean'] *= 1000  # Convert to per-thousand
            summary_df.columns = ['Usage', 'Avg Views', 'Engagement %', 'Sub Rate ‰', 'Score']
            display(summary_df.round(2))
            
        elif analysis_type == 'views':
            fig = create_tag_scatter_analysis(filtered_tag_stats, 'usage_count', 'view_count_mean')
            fig.show()
            
        elif analysis_type == 'engagement':
            fig = create_tag_scatter_analysis(filtered_tag_stats, 'view_count_mean', 'engagement_rate_mean')
            fig.show()
            
        elif analysis_type == 'subscribers':
            fig = create_tag_scatter_analysis(filtered_tag_stats, 'view_count_mean', 'subscriber_rate_mean')
            fig.show()
            
        elif analysis_type == 'usage':
            fig = create_tag_usage_analysis(filtered_tag_df)
            fig.show()
        
        print(f"\nAnalysis complete! Found {len(filtered_tag_stats)} qualifying tags.")

# Connect button to analysis function
controls['update_button'].on_click(lambda x: run_analysis())

# Run initial analysis
run_analysis()

# Display output area
display(output_area)

Output()

In [40]:
# Generate appropriate visualization


## Statistical Insights

In [41]:
def generate_tag_insights(tag_stats, tag_df):
    """Generate statistical insights about tag performance"""
    
    insights = []
    
    # Basic statistics
    total_tags = len(tag_stats)
    total_videos = len(tag_df['video_id'].unique())
    avg_tags_per_video = len(tag_df) / total_videos if total_videos > 0 else 0
    
    insights.append(f"📊 **Dataset Overview:**")
    insights.append(f"   • {total_tags} unique tags analyzed")
    insights.append(f"   • {total_videos} videos with tags")
    insights.append(f"   • {avg_tags_per_video:.1f} average tags per video")
    
    # Best performing tags
    top_effectiveness = tag_stats.head(3)
    insights.append(f"\n🏆 **Top Performing Tags:**")
    for i, (tag, row) in enumerate(top_effectiveness.iterrows(), 1):
        insights.append(
            f"   {i}. **{tag}** - Score: {row['effectiveness_score']:.1f} "
            f"(Used {row['usage_count']} times, {row['view_count_mean']:,.0f} avg views)"
        )
    
    # High-usage vs high-performance analysis
    most_used = tag_stats.nlargest(5, 'usage_count')
    top_performers = tag_stats.nlargest(5, 'effectiveness_score')
    overlap = set(most_used.index) & set(top_performers.index)
    
    insights.append(f"\n🎯 **Usage vs Performance:**")
    insights.append(f"   • {len(overlap)} out of 5 most-used tags are also top performers")
    if len(overlap) < 3:
        insights.append(f"   • Consider using high-performing but underused tags more often")
    
    # Tag effectiveness distribution
    high_effectiveness = (tag_stats['effectiveness_score'] > tag_stats['effectiveness_score'].quantile(0.8)).sum()
    insights.append(f"   • {high_effectiveness} tags ({high_effectiveness/total_tags*100:.1f}%) are highly effective")
    
    # Subscriber acquisition insights
    if 'subscriber_rate_mean' in tag_stats.columns:
        best_sub_tags = tag_stats.nlargest(3, 'subscriber_rate_mean')
        insights.append(f"\n🔔 **Best for Subscriber Acquisition:**")
        for i, (tag, row) in enumerate(best_sub_tags.iterrows(), 1):
            sub_rate = row['subscriber_rate_mean'] * 1000  # Convert to per-thousand
            insights.append(f"   {i}. **{tag}** - {sub_rate:.2f} new subs per 1000 views")
    
    # Engagement insights
    best_engagement = tag_stats.nlargest(3, 'engagement_rate_mean')
    insights.append(f"\n💬 **Best for Engagement:**")
    for i, (tag, row) in enumerate(best_engagement.iterrows(), 1):
        eng_rate = row['engagement_rate_mean'] * 100  # Convert to percentage
        insights.append(f"   {i}. **{tag}** - {eng_rate:.2f}% engagement rate")
    
    # Recommendations
    insights.append(f"\n💡 **Recommendations:**")
    
    # Find underused high-performers
    underused_gems = tag_stats[
        (tag_stats['effectiveness_score'] > tag_stats['effectiveness_score'].quantile(0.7)) & 
        (tag_stats['usage_count'] < tag_stats['usage_count'].quantile(0.3))
    ].head(3)
    
    if len(underused_gems) > 0:
        insights.append(f"   • Consider using these underused high-performers more:")
        for tag in underused_gems.index:
            insights.append(f"     - {tag}")
    
    # Find overused low-performers  
    overused_low = tag_stats[
        (tag_stats['effectiveness_score'] < tag_stats['effectiveness_score'].quantile(0.3)) & 
        (tag_stats['usage_count'] > tag_stats['usage_count'].quantile(0.7))
    ].head(3)
    
    if len(overused_low) > 0:
        insights.append(f"   • Consider reducing usage of these low-performing but frequent tags:")
        for tag in overused_low.index:
            insights.append(f"     - {tag}")
    
    return "\n".join(insights)

# Generate insights for current data
insights_text = generate_tag_insights(tag_stats, tag_df)
print(insights_text)

📊 **Dataset Overview:**
   • 122 unique tags analyzed
   • 91 videos with tags
   • 31.8 average tags per video

🏆 **Top Performing Tags:**
   1. **Revolut** - Score: 70.8 (Used 3.0 times, 29,284 avg views)
   2. **фоп 3 група** - Score: 67.0 (Used 6.0 times, 32,532 avg views)
   3. **фоп** - Score: 64.0 (Used 7.0 times, 29,934 avg views)

🎯 **Usage vs Performance:**
   • 0 out of 5 most-used tags are also top performers
   • Consider using high-performing but underused tags more often
   • 25 tags (20.5%) are highly effective

🔔 **Best for Subscriber Acquisition:**
   1. **Revolut** - 7.60 new subs per 1000 views
   2. **Київстар** - 5.50 new subs per 1000 views
   3. **дія бронювання** - 4.90 new subs per 1000 views

💬 **Best for Engagement:**
   1. **TikTok** - 8.09% engagement rate
   2. **Signal** - 8.00% engagement rate
   3. **як забронюватися від армії** - 7.97% engagement rate

💡 **Recommendations:**
   • Consider using these underused high-performers more:
     - Revolut
    

## Export Analysis Results

In [42]:
def export_analysis_results(tag_stats, filename_suffix=''):
    """Export analysis results to CSV"""
    
    # Create export filename
    from datetime import datetime
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f'../data/exports/tag_analysis_{timestamp}{filename_suffix}.csv'
    
    # Prepare export data
    export_data = tag_stats.copy()
    
    # Convert rates to percentages for better readability
    rate_columns = [col for col in export_data.columns if 'rate' in col and 'normalized' not in col]
    for col in rate_columns:
        export_data[f'{col}_percent'] = export_data[col] * 100
        export_data = export_data.drop(col, axis=1)
    
    # Round numerical columns
    numeric_columns = export_data.select_dtypes(include=[np.number]).columns
    export_data[numeric_columns] = export_data[numeric_columns].round(3)
    
    # Save to CSV
    export_data.to_csv(filename)
    
    print(f"✅ Analysis results exported to: {filename}")
    print(f"📊 Exported {len(export_data)} tags with {len(export_data.columns)} metrics")
    
    return filename

# Export button
export_button = widgets.Button(
    description='Export Results to CSV',
    button_style='success',
    tooltip='Export current analysis to CSV file'
)

def on_export_click(b):
    export_analysis_results(tag_stats)

export_button.on_click(on_export_click)
display(export_button)


Button(button_style='success', description='Export Results to CSV', style=ButtonStyle(), tooltip='Export curre…